# Agentic Retrieval-Augmented Generation (RAG) with Local Llama 2 & ChromaDB

## Overview
This notebook implements an **Agentic Retrieval-Augmented Generation (RAG) pipeline**. It focuses on transcribing audio data, potentially from an Omi streaming device, storing both the transcription and audio, and then using the transcription with a local **Ai Studio** model and **ChromaDB** for intelligent question-answering. The system determines whether additional context is needed before generating responses.

### Key Features:
- **Audio Transcription Workflow** for processing data from devices like Omi.
- **Storage of Audio and Transcriptions** for AI processing.
- **Llama 2 Model** for high-quality text generation.
- **ChromaDB Vector Store** for efficient semantic search on transcriptions.
- **Dynamic Context Retrieval** to improve answer accuracy.
- **Two Answering Modes**:
  - With RAG (Retrieves relevant document content before responding).
  - Without RAG (Directly generates responses).

In [1]:
# Force Python to use the latest system sqlite3 (if available)
import sys
import os

# Unset any pysqlite3 monkeypatching
if "pysqlite3" in sys.modules:
    del sys.modules["pysqlite3"]
if "sqlite3" in sys.modules:
    del sys.modules["sqlite3"]

try:
    import pysqlite3
    sys.modules["sqlite3"] = pysqlite3
    sys.modules["pysqlite3"] = pysqlite3
    print("Using pysqlite3 as sqlite3 backend.")
    sqlite_backend = "pysqlite3"
except ImportError:
    print("pysqlite3 not found, using default sqlite3 module.")
    sqlite_backend = "default"

import sqlite3
print("Loaded sqlite3 version:", sqlite3.sqlite_version)

# Initialize TensorBoard logging for database setup
try:
    from tensorboard_integration import OrpheusTensorBoardManager
    tensorboard_manager = OrpheusTensorBoardManager(
        log_dir="./tensorboard_logs/agentic_rag",
        experiment_name="agentic_rag_audio_pipeline",
        hp_ai_studio_compatible=True
    )
    # Log database configuration
    tensorboard_manager.log_scalar("database_setup/sqlite_version", float(sqlite3.sqlite_version.replace('.', '')[:3])/100, 0)
    print("📊 Database setup metrics logged to TensorBoard")
except ImportError:
    print("⚠️ TensorBoard integration not available yet - will initialize after requirements installation")

Using pysqlite3 as sqlite3 backend.
Loaded sqlite3 version: 3.45.3
✅ TensorBoard integration available
📊 Orpheus Engine TensorBoard Integration Module Loaded
   • TensorBoard Available: ✅
   • Audio Libraries Available: ✅
   • Real-time monitoring ready
   • HP AI Studio integration enabled
📊 TensorBoard writer created: default
   Log directory: tensorboard_logs/agentic_rag/agentic_rag_audio_pipeline/default/20250610-032038
🚀 TensorBoard started successfully!
📊 TensorBoard URL: http://localhost:6006
📁 Log directory: tensorboard_logs/agentic_rag/agentic_rag_audio_pipeline
📊 Database setup metrics logged to TensorBoard


In [2]:
# Install required packages if you have not installed them already
%pip install -r requirements.txt --verbose --quiet
%pip install -q --upgrade pip

# Example output (replace Codespaces path with $HOME):
# Requirement already satisfied: matplotlib>=3.7.0 in $HOME/.local/lib/python3.12/site-packages (from -r requirements.txt (line 21)) (3.10.1)
# Requirement already satisfied: IPython>=8.0.0 in $HOME/.local/lib/python3.12/site-packages (from -r requirements.txt (line 22)) (9.0.2)
# Requirement already satisfied: plotly>=5.15.0 in $HOME/.local/lib/python3.12/site-packages (from -r requirements.txt (line 24)) (6.0.1)
# Requirement already satisfied: pandas>=2.0.0 in $HOME/.local/lib/python3.12/site-packages (from -r requirements.txt (line 25)) (2.2.3)

INFO: pip is looking at multiple versions of dash to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of dash to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
  Using cached pyloudnorm-0.1.1-py3-none-any.whl.metadata (5.6 kB)
  Using cached cython-3.1.2-cp312-cp312-macosx_11_0_arm64.whl.metadata (5.9 kB)
  Using cached tokenizers-0.21.1-cp39-abi3-macosx_11_0_arm64.whl.metadata (6.8 kB)
  Using cached querystring_parser-1.2.4-py2.py3-none-any.whl.metadata (559 bytes)
  Using cached pytz-2024.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached pyarrow-15.0.2-cp312-cp312-macosx_11_0_arm64.whl.metadata (3.0 kB)
  Using cac

In [3]:
# TensorBoard Integration for Real-time Monitoring
# Initialize TensorBoard for monitoring audio transcription and RAG pipeline performance
from tensorboard_integration import OrpheusTensorBoardManager

# Initialize TensorBoard manager for agentic RAG monitoring
tensorboard_manager = OrpheusTensorBoardManager(
    log_dir="./tensorboard_logs/agentic_rag",
    experiment_name="agentic_rag_audio_pipeline",
    hp_ai_studio_compatible=True
)

print("✅ TensorBoard Integration Initialized")
print(f"📊 TensorBoard logs: ./tensorboard_logs/agentic_rag")
print(f"🔍 Experiment: agentic_rag_audio_pipeline")
print(f"🏢 HP AI Studio Compatible: True")
print(f"🌐 View at: http://localhost:6006 (after starting TensorBoard server)")

📊 TensorBoard writer created: default
   Log directory: tensorboard_logs/agentic_rag/agentic_rag_audio_pipeline/default/20250610-032118
🚀 TensorBoard started successfully!
📊 TensorBoard URL: http://localhost:6006
📁 Log directory: tensorboard_logs/agentic_rag/agentic_rag_audio_pipeline
📊 Closed TensorBoard writer: default
✅ TensorBoard Integration Initialized
📊 TensorBoard logs: ./tensorboard_logs/agentic_rag
🔍 Experiment: agentic_rag_audio_pipeline
🏢 HP AI Studio Compatible: True
🌐 View at: http://localhost:6006 (after starting TensorBoard server)


## 🔧 Step 1: Model Setup

We will set up **Llama 2 (7B)** for text generation. If the model is not found locally, it will be downloaded from Hugging Face.

In [4]:
# Log model setup metrics to TensorBoard
tensorboard_manager.log_scalar("model_setup/model_size_gb", 3.7, 0)  # Approximate Llama 2 7B size
tensorboard_manager.log_scalar("model_setup/context_length", 4096, 0)
tensorboard_manager.log_scalar("model_setup/max_tokens", 2000, 0)
tensorboard_manager.log_scalar("model_setup/temperature", 0.25, 0)
tensorboard_manager.log_scalar("model_setup/gpu_layers", 30, 0)

print("📊 Model setup metrics logged to TensorBoard")

📊 Model setup metrics logged to TensorBoard


In [5]:
%pip install -q huggingface-hub

import os
from huggingface_hub import hf_hub_download

MODEL_FILENAME = "llama-2-7b-chat.Q4_K_M.gguf"
MODEL_DIR = "model"
EXPECTED_PATH = os.path.join(MODEL_DIR, MODEL_FILENAME)

# Ensure model directory exists
os.makedirs(MODEL_DIR, exist_ok=True)

# Check if model already exists
if os.path.exists(EXPECTED_PATH):
    print(f"Model already exists at: {EXPECTED_PATH}")
    model_path = EXPECTED_PATH
else:
    print("Model not found locally. Downloading Llama 2 model...")
    
    # Download the model - Fixed: removed url parameter and added correct parameters
    model_path = hf_hub_download(
        repo_id="TheBloke/Llama-2-7B-Chat-GGUF",
        filename=MODEL_FILENAME,
        local_dir=MODEL_DIR
    )
    print(f"Model downloaded to: {model_path}")

print(f"Using model at: {model_path}")

Note: you may need to restart the kernel to use updated packages.
Model not found locally. Downloading Llama 2 model...


llama-2-7b-chat.Q4_K_M.gguf:   0%|          | 0.00/4.08G [00:00<?, ?B/s]

Model downloaded to: model/llama-2-7b-chat.Q4_K_M.gguf
Using model at: model/llama-2-7b-chat.Q4_K_M.gguf


In [6]:
%pip install -q llama-cpp-python
# Check if the model file exists
if not os.path.exists(model_path):
    raise FileNotFoundError(f"Model file not found at {model_path}")

# Import the Llama class from llama_cpp
from llama_cpp import Llama

# Initialize the model with the local path and GPU acceleration
llm = Llama(
    model_path=EXPECTED_PATH,
    temperature=0.25,
    max_tokens=2000,
    n_ctx=4096,
    top_p=1.0,
    verbose=False,
    n_gpu_layers=30,  # Utilize some available GPU layers
    n_batch=512,      # Optimize batch size for parallel processing
    f16_kv=True,      # Enable half-precision for key/value cache
    use_mlock=True,   # Lock memory to prevent swapping
    use_mmap=True     # Utilize memory mapping for faster loading
)

Note: you may need to restart the kernel to use updated packages.


ggml_metal_init: skipping kernel_get_rows_bf16                     (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_f32                   (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_f32_1row              (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_f32_l4                (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_bf16                  (not supported)
ggml_metal_init: skipping kernel_mul_mv_id_bf16_f32                (not supported)
ggml_metal_init: skipping kernel_mul_mm_bf16_f32                   (not supported)
ggml_metal_init: skipping kernel_mul_mm_id_bf16_f32                (not supported)
ggml_metal_init: skipping kernel_flash_attn_ext_bf16_h64           (not supported)
ggml_metal_init: skipping kernel_flash_attn_ext_bf16_h80           (not supported)
ggml_metal_init: skipping kernel_flash_attn_ext_bf16_h96           (not supported)
ggml_metal_init: skipping kernel_flash_attn_ext_bf16_h112          (not supported)
ggml

## 📄 Step 2: Loading, Transcribing, and Storing Audio Data

This step outlines the process for loading audio data (e.g., from an Omi streaming device), transcribing it, and preparing it for storage and further processing. Both the raw audio and its transcription are valuable assets.

### Updated Workflow:
- Load audio data from files or streams.
- Transcribe audio to text using the LLM model.
- Extract and tag audio segments based on content.
- Store audio and transcription in a structured format.
- (Optional) Save audio segments as separate files for download or sharing.

In [7]:
# --- Load the Audio File Document ---
# --- Load the Audio File Document and Audio Collection System ---

# Install whisper if not already installed
%pip install -q openai-whisper

# Install ffmpeg-python bindings if not already installed
%pip install -q ffmpeg-python

import shutil
import sys
import subprocess

# Check if ffmpeg is available and working
try:
    subprocess.run(["ffmpeg", "-version"], capture_output=True, check=True)
    print("ffmpeg found and working.")
except FileNotFoundError:
    raise FileNotFoundError("ffmpeg not found. Please install ffmpeg and ensure it's in your PATH.")
except subprocess.CalledProcessError as e:
    print(f"Error running ffmpeg: {{e}}")
    raise RuntimeError("ffmpeg is not working correctly.") from e

import whisper

# No need to manually set ffmpeg_dir if ffmpeg is installed system-wide

# Define the Audio File file path
AUDIO_PATH = "./data/tester.mp3"
# Check if the file exists
print(f"Loading AUDIO from: {AUDIO_PATH}")

# Define the audio collection system
AUDIO_COLLECTION_SYSTEM = "MyAudioSystem"

# TensorBoard logging for audio transcription performance
import time
start_time = time.time()

# Load and transcribe the audio file using whisper
model = whisper.load_model("base")
result = model.transcribe(AUDIO_PATH)
text_content = result["text"]

# Calculate and log transcription metrics
transcription_time = time.time() - start_time
audio_duration = result.get("duration", 0)
segments_count = len(result.get("segments", []))

# Log metrics to TensorBoard
if 'tensorboard_manager' in locals():
    tensorboard_manager.log_scalar("audio_transcription/processing_time", transcription_time, 0)
    tensorboard_manager.log_scalar("audio_transcription/audio_duration", audio_duration, 0)
    tensorboard_manager.log_scalar("audio_transcription/segments_count", segments_count, 0)
    tensorboard_manager.log_scalar("audio_transcription/processing_speed_ratio", audio_duration / transcription_time if transcription_time > 0 else 0, 0)
    print(f"📊 Transcription metrics logged to TensorBoard")

print(f"🎧 Audio transcribed in {transcription_time:.2f}s (duration: {audio_duration:.2f}s)")

# Log audio quality metrics if available
if "language" in result:
    print(f"🌍 Detected language: {result['language']}")
    if 'tensorboard_manager' in locals():
        # Log language detection success
        tensorboard_manager.log_scalar("audio_transcription/language_detected", 1.0, 0)
    
print(f"📦 Extracted {segments_count} segments for processing")

# For compatibility with the rest of your code, wrap the text in a document-like object
class AudioDocument:
    def __init__(self, text):
        self.page_content = text
        self.metadata = {}  # Optionally add metadata if needed
    def getPageText(self):
        return self.page_content

documents = [AudioDocument(text_content)]

print(f"Successfully loaded {len(documents)} document(s) from the AUDIO.")
# Initialize an empty list for the audio files
audio_files = []

# Iterate through each document
for document in documents:
    # Get the current page's text content
    text_content = document.getPageText()

    # Extract relevant information from the text, e.g., keywords or phrases
    def extractRelevantInfo(text):
        # Placeholder: just return the text itself
        return text
    extracted_info = extractRelevantInfo(text_content)

    # Define a placeholder for createAudioFile
    def createAudioFile(system, info):
        # Placeholder: just return a tuple for demonstration
        return (system, info)

    # Create an audio file based on the extracted information
    audio_file = createAudioFile(AUDIO_COLLECTION_SYSTEM, extracted_info)

    # Add the audio file to the collection system's list
    audio_files.append(audio_file)
print(f"Successfully loaded {len(documents)} document(s) from the AUDIO and created {len(audio_files)} audio file(s).")

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
ffmpeg found and working.
Loading AUDIO from: ./data/tester.mp3


/Volumes/PRO-BLADE/opt/anaconda3/lib/python3.12/site-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


RuntimeError: Failed to load audio: ffmpeg version 7.1.1 Copyright (c) 2000-2025 the FFmpeg developers
  built with Apple clang version 16.0.0 (clang-1600.0.26.6)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/7.1.1_3 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags='-Wl,-ld_classic' --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libharfbuzz --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --

In [ ]:
# Get Omi data from Webhook

# Initialize variables for Omi data from Webhook
response = None  # Will store webhook response
rag_percentage = 0.0  # Initialize RAG percentage
ipfs_metrics = {}  # Initialize IPFS metrics dictionary

# --- Audio Event Detection (Sound Tagging) ---

%pip install -q torchaudio
%pip install -q panns-inference

import torch
import torchaudio
from panns_inference import AudioTagging

# Initialize the PANNs audio tagging model
panns_model = AudioTagging(checkpoint_path=None, device='cpu')  # Uses default Cnn14 weights

# Load and preprocess audio
waveform, sr = torchaudio.load(AUDIO_PATH)
if sr != 32000:
    waveform = torchaudio.transforms.Resample(orig_freq=sr, new_freq=32000)(waveform)
    sr = 32000

# PANNs expects mono audio
if waveform.shape[0] > 1:
    waveform = torch.mean(waveform, dim=0, keepdim=True)

# --- Load the full AudioSet class labels for PANNs output mapping ---
import os
import csv

LABELS_CSV_PATH = "class_labels_indices.csv"
labels = None
if os.path.exists(LABELS_CSV_PATH):
    with open(LABELS_CSV_PATH, newline='') as csvfile:
        reader = csv.DictReader(csvfile)
        labels = [row['display_name'] for row in reader]
else:
    print("WARNING: AudioSet class label CSV not found. Will print indices instead of class names.")

# Perform inference
with torch.no_grad():
    output = panns_model.inference(waveform)
    clipwise_output = output[0]
    topk = torch.topk(torch.tensor(clipwise_output[0]), 3)
    sound_tags = []
    if labels and max(topk.indices.tolist()) < len(labels):
        sound_tags = [labels[i] for i in topk.indices.tolist()]
    else:
        sound_tags = [f"Class index {i}" for i in topk.indices.tolist()]

print("Detected sound types:", sound_tags)

# Example output (replace Codespaces path with $HOME):
# Checkpoint path: $HOME/panns_data/Cnn14_mAP=0.431.pth

## ✂️ Step 3: Chunking Audio Transcriptions for RAG

The transcribed text from the audio data is split into **small overlapping chunks** (approximately **500 characters**). These chunks are then used for embedding and storage in ChromaDB to enable semantic search for the RAG pipeline.

### Updated Workflow:
- Split transcriptions into smaller chunks to manage context size.
- Overlap chunks slightly to ensure continuity and context preservation.
- Tag chunks with relevant metadata (e.g., sound types, timestamps).
- Store chunks in ChromaDB with embeddings for efficient retrieval.

In [ ]:
# --- Split the Audio Content into Manageable Chunks with Sound Tags ---
%pip install -q langchain

from langchain.text_splitter import CharacterTextSplitter

CHUNK_SIZE = 500
CHUNK_OVERLAP = 50
text_splitter = CharacterTextSplitter(chunk_size=CHUNK_SIZE, chunk_overlap=CHUNK_OVERLAP)


# Split the transcription into chunks
docs = text_splitter.split_documents(documents)

# Attach sound tags to each chunk as metadata
for doc in docs:
    doc.page_content = f"Transcription: {doc.page_content}\nSound tags: {', '.join(sound_tags)}"

## 🔍 Step 4: Initializing the Embedding Model

To convert text into numerical representations for efficient similarity search, we use **all-MiniLM-L6-v2** from `sentence-transformers`.

In [ ]:
# --- Initialize the Embedding Model ---
%pip install -q sentence-transformers
%pip install -q transformers

from sentence_transformers import SentenceTransformer

# Define the embedding model name
MODEL_NAME = "all-MiniLM-L6-v2"

# Load the embedding model
embedding_model = SentenceTransformer(MODEL_NAME)

print(f"Successfully loaded embedding model: {MODEL_NAME}")

## 🧠 Step 5: Computing Embeddings for Document Chunks

Each chunk is converted into a **vector representation** using our embedding model. This allows us to perform **semantic similarity searches** later.

In [ ]:
# --- Compute Embeddings for Each Text Chunk ---

# Extract text content from each chunk
doc_texts = [doc.page_content for doc in docs]

# Compute embeddings for the extracted text chunks
document_embeddings = embedding_model.encode(doc_texts, convert_to_numpy=True)

# Display the result
print("Successfully computed embeddings for each text chunk.")
print(f"Embeddings Shape: {document_embeddings.shape}")

## 🗄️ Step 6: Storing Audio Transcription Embeddings in ChromaDB

We initialize **ChromaDB**, a high-performance **vector database**, and store our computed embeddings to enable efficient retrieval of relevant text chunks.

In [ ]:
# --- Initialize and Populate the Chroma Vector Database ---

# Define Chroma database path and collection name
CHROMA_DB_PATH = "./chroma_db"
COLLECTION_NAME = "document_embeddings"

# Initialize Chroma client
import chromadb
chroma_client = chromadb.PersistentClient(path=CHROMA_DB_PATH)
collection = chroma_client.get_or_create_collection(name=COLLECTION_NAME)

# Add document embeddings to the Chroma collection
for i, embedding in enumerate(document_embeddings):
    collection.add(
        ids=[str(i)],  # Chroma requires string IDs
        embeddings=[embedding.tolist()],
        metadatas=[{"text": doc_texts[i]}]
    )

print("Successfully populated Chroma database with document embeddings.")

## 🔎 Step 7: Implementing Vector Search Tool

To retrieve relevant text passages from the database, we define a **vector search function** that finds the most relevant chunks based on a user query.

In [ ]:
def vector_search_tool(query: str) -> str:
    """
    Searches the Chroma database for relevant text chunks based on the query.
    Computes the query embedding, retrieves the top 5 most relevant text chunks,
    and returns them as a formatted string.
    """
    query_embedding = embedding_model.encode(query, convert_to_numpy=True).tolist()
    TOP_K = 5
    results = collection.query(
        query_embeddings=[query_embedding],
        n_results=TOP_K
    )
    retrieved_chunks = [metadata["text"] for metadata in results["metadatas"][0]]
    return "\n\n".join(retrieved_chunks)

print(vector_search_tool("example query"))  # Test the search tool with a sample query

## 🤖 Step 8: Context Need Assessment

Instead of always retrieving context, we determine if the query **requires external document context** before generating a response. This creates an agentic workflow that makes autonomous decisions to complete the task at hand.

In [ ]:
import torchaudio
import re

# --- Audio Segment Retrieval and Saving ---
def find_relevant_audio_segments(query, segments, collection, embedding_model, top_k=5):
    """
    Retrieve audio segments whose text contains the query string (case-insensitive substring match).
    Args:
        query (str): The user query.
        segments (list): List of segment dicts with 'text', 'start', 'end', 'id'.
        collection: ChromaDB collection (not used in this version).
        embedding_model: Embedding model (not used in this version).
        top_k (int): Not used in this version.
    Returns:
        List of relevant segment dicts.
    """
    query_lower = query.lower()
    relevant_segments = [segment for segment in segments if query_lower in segment["text"].lower()]
    print(f"Total relevant segments found: {len(relevant_segments)}")
    return relevant_segments

# Function to sanitize filenames
def sanitize_filename(name):
    return re.sub(r'[\\/*?:"<>|]', "_", name).strip()

# Function to save audio segments
def save_audio_segments(audio_path, segments, query, output_base="audio_clips"):
    """
    Save relevant audio segments as separate files.
    """
    folder_name = sanitize_filename(query)
    output_dir = os.path.join(output_base, folder_name)
    os.makedirs(output_dir, exist_ok=True)
    waveform, sr = torchaudio.load(audio_path)
    for seg in segments:
        start_sample = int(seg['start'] * sr)
        end_sample = int(seg['end'] * sr)
        clip_waveform = waveform[:, start_sample:end_sample]
        out_path = os.path.join(output_dir, f"segment_{{seg['id']}}_{{int(seg['start'])}}-{{int(seg['end'])}}.wav")
        torchaudio.save(out_path, clip_waveform, sr)
        print(f"Saved: {out_path}")

# Example usage:
# segments = result["segments"]  # Ensure you have this from the Whisper transcription
# relevant_segments = find_relevant_audio_segments("audio", segments, collection, embedding_model)
# save_audio_segments(AUDIO_PATH, relevant_segments, "audio")
# for seg in relevant_segments:
#     print(f"Start: {seg['start']}s, End: {seg['end']}s, Text: {seg['text']}")

## 💡 Step 9: update OEW-MAIN audio library with the audio created from the search.

This step will integrate the audio files created or identified in this pipeline into the main audio library of the OEW system. This ensures that all processed and relevant audio content is available for the user's projects and can be easily accessed or downloaded.

## 🎚️ Step 10: DAW Integration & Audio Rendering

Now that relevant audio segments have been extracted, we can integrate them into a Digital Audio Workstation (DAW) environment. This step demonstrates how to load, visualize, and play back audio clips, enabling further editing or composition. Below, we render audio waveforms and provide playback controls for the extracted segments.

In [ ]:
# --- Render and Play Extracted Audio Segments in Notebook ---

import torchaudio
import matplotlib.pyplot as plt
import IPython.display as ipd
import os

def render_audio_segments(folder_path):
    """
    Display waveforms and playback controls for all audio clips in the given folder.
    """
    audio_files = [f for f in os.listdir(folder_path) if f.endswith(".wav")]
    if not audio_files:
        print("No audio clips found in:", folder_path)
        return
    for audio_file in sorted(audio_files):
        file_path = os.path.join(folder_path, audio_file)
        waveform, sr = torchaudio.load(file_path)
        plt.figure(figsize=(10, 2))
        plt.title(audio_file)
        plt.plot(waveform.t().numpy())
        plt.xlabel("Sample")
        plt.ylabel("Amplitude")
        plt.show()
        display(ipd.Audio(file_path))
        print("-" * 40)

# Example usage: visualize and play audio clips for the last query
render_audio_segments(os.path.join("audio_clips", sanitize_filename("audio")))

## 📤 Step 11: Exporting Audio Clips

After rendering and reviewing audio segments, you may want to export them for sharing, further processing, or integration with other systems. Below are common export options:
- **IPFS**: Upload audio to the InterPlanetary File System for decentralized access.
- **AWS S3**: Store audio in an Amazon S3 bucket for scalable cloud storage.
- **Direct Download**: Save audio to a temporary folder for local or web-based download.

In [ ]:
# --- Export Audio Clips: IPFS, AWS S3, or Direct Download ---

import os
import shutil
import tempfile

# Optional: install required packages for IPFS and AWS S3
# %pip install ipfshttpclient boto3

def export_audio_clips(folder_path, method="direct", **kwargs):
    """
    Export audio clips using the specified method.
    method: "ipfs", "aws", or "direct"
    kwargs: Additional arguments for each method.
    Returns a list of export URLs or file paths.
    """
    audio_files = [f for f in os.listdir(folder_path) if f.endswith(".wav")]
    exported = []
    if method == "ipfs":
        # Example: Upload to IPFS (requires ipfshttpclient)
        import ipfshttpclient
        client = ipfshttpclient.connect()
        for audio_file in audio_files:
            file_path = os.path.join(folder_path, audio_file)
            res = client.add(file_path)
            ipfs_url = f"https://ipfs.io/ipfs/{{res['Hash']}}"
            exported.append(ipfs_url)
            print(f"Exported to IPFS: {{ipfs_url}}")
    elif method == "aws":
        # Example: Upload to AWS S3 (requires boto3)
        import boto3
        s3 = boto3.client("s3")
        bucket = kwargs.get("bucket")
        prefix = kwargs.get("prefix", "")
        for audio_file in audio_files:
            file_path = os.path.join(folder_path, audio_file)
            s3_key = os.path.join(prefix, audio_file)
            s3.upload_file(file_path, bucket, s3_key)
            s3_url = f"https://{{bucket}}.s3.amazonaws.com/{{s3_key}}"
            exported.append(s3_url)
            print(f"Exported to S3: {{s3_url}}")
    elif method == "direct":
        # Copy files to a temporary directory for download
        temp_dir = tempfile.mkdtemp(prefix="audio_export_")
        for audio_file in audio_files:
            src = os.path.join(folder_path, audio_file)
            dst = os.path.join(temp_dir, audio_file)
            shutil.copy2(src, dst)
            exported.append(dst)
            print(f"Copied to temp folder: {{dst}}")
        print(f"All files available in: {{temp_dir}}")
    else:
        raise ValueError("Unknown export method.")
    return exported

# Example usage:
# Export to IPFS (requires running IPFS daemon and ipfshttpclient)
# export_audio_clips(os.path.join("audio_clips", sanitize_filename("audio")), method="ipfs")

# Export to AWS S3 (requires AWS credentials)
# export_audio_clips(os.path.join("audio_clips", sanitize_filename("audio")), method="aws", bucket="your-bucket", prefix="exports/")

# Export for direct download (local temp folder)
export_audio_clips(os.path.join("audio_clips", sanitize_filename("audio")), method="direct")

## 📊 Step 12: RAG Pipeline Monitoring Dashboard

To efficiently track and monitor our audio processing pipeline, we'll implement a simple web dashboard that provides:

- **Audio Processing Metrics**: Track transcription times, file sizes, and segment counts
- **ChromaDB Interactions**: Monitor query frequency, embedding operations, and retrieval times 
- **RAG Performance**: Visualize context usage decisions, response generation times, and user queries

This dashboard runs on `http://127.0.0.1:5001/` and provides real-time insights into the operation of our pipeline.

In [ ]:
# --- Implement RAG Pipeline Monitoring Dashboard ---
%pip install -q streamlit plotly pandas

import streamlit as st
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import threading
import datetime
import time
import json
import socket
from collections import defaultdict, deque

# Function to find an available port starting from a given port
def find_available_port(start_port):
    port = start_port
    max_port = start_port + 100  # Try 100 ports at most
    
    while port < max_port:
        with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
            try:
                s.bind(('127.0.0.1', port))
                return port
            except OSError:
                port += 1
    
    raise RuntimeError(f"Could not find available port starting from {start_port}")

# Create a class to track metrics across the entire pipeline
class RAGMetricsTracker:
    def __init__(self, max_history=100):
        self.audio_metrics = {
            "processed_files": 0,
            "total_duration": 0,
            "avg_transcription_time": 0,
            "segments_extracted": 0,
            "history": deque(maxlen=max_history)
        }
        self.chromadb_metrics = {
            "queries": 0,
            "embeddings_created": 0,
            "avg_query_time": 0,
            "query_times": deque(maxlen=max_history)
        }
        self.rag_metrics = {
            "total_queries": 0,
            "context_used": 0,  # Number of times RAG used additional context
            "direct_answers": 0,  # Number of times RAG answered directly
            "query_history": deque(maxlen=max_history)
        }
        self.timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    
    def log_audio_process(self, filename, duration, transcription_time, num_segments):
        



        self.audio_metrics["processed_files"] += 1
        self.audio_metrics["total_duration"] += duration
        
        # Update running average transcription time
        prev_avg = self.audio_metrics["avg_transcription_time"]
        prev_count = self.audio_metrics["processed_files"] - 1
        self.audio_metrics["avg_transcription_time"] = (prev_avg * prev_count + transcription_time) / self.audio_metrics["processed_files"]
        
        self.audio_metrics["segments_extracted"] += num_segments
        
        # Add to history
        self.audio_metrics["history"].append({
            "timestamp": datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
            "filename": filename,
            "duration": duration,
            "transcription_time": transcription_time,
            "segments": num_segments
        })
    
    def log_chromadb_query(self, query, num_results, query_time):
        



        self.chromadb_metrics["queries"] += 1
        self.chromadb_metrics["query_times"].append(query_time)
        self.chromadb_metrics["avg_query_time"] = sum(self.chromadb_metrics["query_times"]) / len(self.chromadb_metrics["query_times"])
    
    def log_embedding_creation(self, count):
        



        self.chromadb_metrics["embeddings_created"] += count
    
    def log_user_query(self, query, used_context, response_time):
        



        self.rag_metrics["total_queries"] += 1
        if used_context:
            self.rag_metrics["context_used"] += 1
        else:
            self.rag_metrics["direct_answers"] += 1
            
        self.rag_metrics["query_history"].append({
            "timestamp": datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
            "query": query,
            "used_context": used_context,
            "response_time": response_time
        })
    
    def export_metrics(self):
        



        return {
            "audio_metrics": {
                "processed_files": self.audio_metrics["processed_files"],
                "total_duration": self.audio_metrics["total_duration"],
                "avg_transcription_time": self.audio_metrics["avg_transcription_time"],
                "segments_extracted": self.audio_metrics["segments_extracted"],
                "history": list(self.audio_metrics["history"])
            },
            "chromadb_metrics": {
                "queries": self.chromadb_metrics["queries"],
                "embeddings_created": self.chromadb_metrics["embeddings_created"],
                "avg_query_time": self.chromadb_metrics["avg_query_time"],
                "query_times": list(self.chromadb_metrics["query_times"])
            },
            "rag_metrics": {
                "total_queries": self.rag_metrics["total_queries"],
                "context_used": self.rag_metrics["context_used"],
                "direct_answers": self.rag_metrics["direct_answers"],
                "query_history": list(self.rag_metrics["query_history"])
            },
            "timestamp": self.timestamp
        }

# Find available ports for our services
metrics_port = find_available_port(5002)
dashboard_port = find_available_port(5001)
print(f"Selected ports: Dashboard={{dashboard_port}}, Metrics API={{metrics_port}}")

# Create a global metrics tracker instance
metrics_tracker = RAGMetricsTracker()

# Retroactively log the metrics from our previous operations
metrics_tracker.log_audio_process(
    filename=AUDIO_PATH,
    duration=result["segments"][-1]["end"] if len(result["segments"]) > 0 else 0,
    transcription_time=3.5,  # Placeholder value
    num_segments=len(result["segments"])
)

metrics_tracker.log_embedding_creation(len(document_embeddings))

# Wrap our vector search function to track metrics
original_vector_search = vector_search_tool

def tracked_vector_search_tool(query: str) -> str:
    start_time = time.time()
    result = original_vector_search(query)
    query_time = time.time() - start_time
    metrics_tracker.log_chromadb_query(query, 5, query_time)
    return result

# Replace the original function with our tracked version
vector_search_tool = tracked_vector_search_tool

# Create the Streamlit dashboard
def create_dashboard():
    """
    Create the Streamlit dashboard for RAG pipeline monitoring
    """
    
    # Define the Streamlit app with dynamic port for metrics API
    dashboard_code = f"""
import streamlit as st
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import json
import time
import requests

st.set_page_config(page_title="RAG Pipeline Monitor", layout="wide")

st.title("📊 Agentic RAG Pipeline Monitoring")

# Create metrics columns
col1, col2, col3, col4 = st.columns(4)

# Initialize session state
if 'last_update' not in st.session_state:
    st.session_state.last_update = time.time()
    st.session_state.metrics_history = []

# Function to fetch metrics from the metrics endpoint
def get_metrics():
    try:
        response = requests.get(METRICS_API_URL)
        if response.status_code == 200:
            return response.json()
        else:
            st.error(f"Failed to fetch metrics: {response.status_code}")
            return None
    except Exception as e:
        st.error(f"Error fetching metrics: {e}")
        return None

# Set the metrics API endpoint
METRICS_API_URL = "http://127.0.0.1:METRICS_PORT/metrics"

metrics = get_metrics()
if metrics:
    # Store metrics in history
    st.session_state.metrics_history.append(metrics)
    if len(st.session_state.metrics_history) > 20:
        st.session_state.metrics_history.pop(0)
    
    # Display metrics in columns
    with col1:
        st.subheader("Audio Processing")
        st.metric("Files Processed", metrics["audio_metrics"]["processed_files"])
        st.metric("Total Duration (sec)", round(metrics["audio_metrics"]["total_duration"], 2))
        st.metric("Segments Extracted", metrics["audio_metrics"]["segments_extracted"])
    
    with col2:
        st.subheader("ChromaDB Operations")
        st.metric("Total Queries", metrics["chromadb_metrics"]["queries"])
        st.metric("Embeddings Created", metrics["chromadb_metrics"]["embeddings_created"])
        st.metric("Avg Query Time (ms)", round(metrics["chromadb_metrics"]["avg_query_time"] * 1000, 2))
    
    with col3:
        st.subheader("RAG Performance")
        st.metric("User Queries", metrics["rag_metrics"]["total_queries"])
        rag_usage = metrics["rag_metrics"]["context_used"]
        direct_answers = metrics["rag_metrics"]["direct_answers"]
        total = rag_usage + direct_answers
        
        if total > 0:
            rag_percentage = (rag_usage / total) * 100
            st.metric("RAG Usage", f"{rag_percentage:.1f}%")
        else:
            st.metric("RAG Usage", "0%")
    
    # New IPFS metrics column
    with col4:
        st.subheader("IPFS Integration")
        if "ipfs_metrics" in metrics:
            ipfs_metrics = metrics["ipfs_metrics"]
            st.metric("Files Discovered", ipfs_metrics["audio_files_discovered"])
            st.metric("Files Processed", ipfs_metrics["audio_files_processed"])
            
            # Show search info if available
            if ipfs_metrics["last_search_query"]:
                st.text(f"Last search: {ipfs_metrics['last_search_query']}")
                st.text(f"Results: {ipfs_metrics['last_search_results']}")
        else:
            st.text("No IPFS metrics available")
    
    # Time series chart of audio processing
    st.subheader("Audio Processing History")
    if metrics["audio_metrics"]["history"]:
        audio_df = pd.DataFrame(metrics["audio_metrics"]["history"])
        if not audio_df.empty:
            audio_df['timestamp'] = pd.to_datetime(audio_df['timestamp'])
            fig = px.scatter(audio_df, x='timestamp', y='transcription_time', 
                             size='duration', color='segments',
                             title="Audio Transcription Performance")
            st.plotly_chart(fig, use_container_width=True)
    
    # Query visualization
    st.subheader("Recent User Queries")
    if metrics["rag_metrics"]["query_history"]:
        query_df = pd.DataFrame(metrics["rag_metrics"]["query_history"])
        if not query_df.empty:
            query_df['timestamp'] = pd.to_datetime(query_df['timestamp'])
            query_df['used_context_label'] = query_df['used_context'].map({True: "RAG", False: "Direct"})
            
            fig = px.scatter(query_df, x='timestamp', y='response_time',
                            color='used_context_label', hover_data=['query'],
                            title="Query Response Times")
            st.plotly_chart(fig, use_container_width=True)
            
            # Display recent queries in a table
            st.dataframe(query_df[['timestamp', 'query', 'used_context_label', 'response_time']]
                        .rename(columns={{'used_context_label': 'Method', 'response_time': 'Time (sec)'}}))
    
    st.text(f"Last updated: {{time.strftime('%Y-%m-%d %H:%M:%S')}}")
    st.text(f"Dashboard port: {{dashboard_port}}, Metrics API port: {{metrics_port}}")
    
    # Auto-refresh every 5 seconds
    time.sleep(5)
    st.experimental_rerun()
else:
    st.warning("Failed to fetch metrics data. Make sure the metrics endpoint is running.")
    st.text(f"Looking for metrics at: {{METRICS_API_URL}}")
    time.sleep(5)
    st.experimental_rerun()
"""
    
    # Write the Streamlit app to a file
    with open("rag_dashboard.py", "w") as f:
        f.write(dashboard_code)
    
    # Create a simple Flask server to serve the metrics with dynamic port
    flask_code = f"""
from flask import Flask, jsonify
import json
import os

app = Flask(__name__)

@app.route('/metrics')
def get_metrics():
    try:
        with open('rag_metrics.json', 'r') as f:
            metrics = json.load(f)
        return jsonify(metrics)
    except Exception as e:
        return jsonify({{"error": str(e)}}), 500

if __name__ == '__main__':
    app.run(host='127.0.0.1', port={metrics_port})
"""
    
    with open("metrics_server.py", "w") as f:
        f.write(flask_code)
    
    # Export the current metrics to a JSON file
    with open("rag_metrics.json", "w") as f:
        json.dump(metrics_tracker.export_metrics(), f)
    
    # Start the flask server in a separate thread
    def run_flask():
        os.system("python metrics_server.py")
    
    flask_thread = threading.Thread(target=run_flask)
    flask_thread.daemon = True
    flask_thread.start()
    
    # Start the Streamlit dashboard with the selected port
    print("Starting RAG pipeline dashboard...")
    os.system(f"streamlit run rag_dashboard.py --server.port={{dashboard_port}}")

# Run the dashboard in a separate thread to avoid blocking the notebook
dashboard_thread = threading.Thread(target=create_dashboard)
dashboard_thread.daemon = True
dashboard_thread.start()

print(f"RAG pipeline monitoring dashboard is starting...")
print(f"Access the dashboard at http://localhost:{{dashboard_port}}")
print(f"Metrics API running at http://localhost:{{metrics_port}}/metrics")

# Example of RAG metrics tracking:
print("\nExample of RAG metrics tracking:")
print("--------------------------------")

# Example of tracking a query to demonstrate the logging
def simulate_rag_query(query, use_context=True):
    start_time = time.time()
    
    if use_context:
        context = tracked_vector_search_tool(query)
        # Simulate RAG processing with context
        time.sleep(0.5)  # Simulate processing time
    else:
        # Simulate direct answer without context
        time.sleep(0.2)  # Simulate faster processing
    
    response_time = time.time() - start_time
    metrics_tracker.log_user_query(query, use_context, response_time)
    
    # Export updated metrics
    with open("rag_metrics.json", "w") as f:
        json.dump(metrics_tracker.export_metrics(), f)
    
    return "Response time: {:.2f}s, Context used: {}".format(response_time, use_context)

# Simulate a few queries with and without context
print(simulate_rag_query("What are the main topics discussed in the audio?", use_context=True))
print(simulate_rag_query("Who is speaking in the audio?", use_context=True))
print(simulate_rag_query("What is the background noise?", use_context=False))
print(simulate_rag_query("When was this recording made?", use_context=False))

print(f"\nOpen the dashboard at http://localhost:{{dashboard_port}} to see these metrics visualized")

## 🔍 Step 13: IPFS Audio Content Discovery and Processing

This step implements functionality to discover and retrieve audio content stored on IPFS by examining metadata. This allows the AI to expand its knowledge base by incorporating audio content from decentralized storage systems, effectively giving it access to a wider range of audio sources beyond local files.

In [ ]:
# --- IPFS Audio Content Discovery and Processing ---

%pip install -q ipfshttpclient requests

import ipfshttpclient
import requests
import json
import tempfile
import os
from urllib.parse import urlparse, urljoin

def search_ipfs_audio_content(gateway_url="https://ipfs.io/api/v0", query=None, limit=10):
    """
    Search for audio content on IPFS using metadata
    
    Args:
        gateway_url: IPFS gateway URL
        query: Optional search terms
        limit: Maximum number of results to return
        
    Returns:
        List of dicts containing CIDs and metadata for audio files
    """
    print(f"Searching IPFS for audio content{f' related to {query}' if query else ''}...")
    audio_files = []
    
    try:
        # Try to connect to a local IPFS daemon first
        try:
            client = ipfshttpclient.connect()
            print("Connected to local IPFS daemon")
        except:
            print("No local IPFS daemon found, using gateway API")
            client = None
            
        # Method 1: If we have a local client, use it to search the DHT
        if client:
            # Get list of pins if no query provided
            if not query:
                pins = client.pin.ls(type="recursive")
                for pin in pins["Keys"]:
                    try:
                        # Get metadata for each pin
                        metadata = client.object.get(pin)
                        if is_audio_by_metadata(metadata):
                            audio_files.append({
                                "cid": pin,
                                "metadata": metadata,
                                "source": "local_ipfs"
                            })
                            if len(audio_files) >= limit:
                                break
                    except Exception as e:
                        print(f"Error examining pin {pin}: {e}")
            else:
                # If query provided, search IPNS/IPFS links containing audio-related terms
                # This is simplified as full DHT search requires more complex code
                pass
        
        # Method 2: Use IPFS gateway API or a pinning service
        # This is a simplified implementation that would need a specific gateway API supporting search
        if query and len(audio_files) < limit:
            search_url = f"{gateway_url}/search?q={query}+audio+filetype:mp3+filetype:wav"
            try:
                response = requests.get(search_url, timeout=10)
                if response.status_code == 200:
                    results = response.json()
                    for result in results:
                        if is_audio_by_metadata(result.get("metadata", {})):
                            audio_files.append({
                                "cid": result.get("cid"),
                                "metadata": result.get("metadata", {}),
                                "source": "gateway"
                            })
                            if len(audio_files) >= limit:
                                break
            except Exception as e:
                print(f"Error searching gateway: {e}")
                
        # Fallback: Use known audio CIDs for demonstration
        if len(audio_files) == 0:
            print("No audio files found via search, using example files for demonstration")
            # These are example CIDs - they would need to be replaced with actual audio CIDs
            example_audio_cids = [
                "QmYwAPJzv5CZsnA625s3Xf2nemtYgPpHdWEz79ojWnPbdG/demo.mp3",
                "QmZ4tDuvesekSs4qM5ZBKpXiZGun7S2CYtEZRB3DYXkjGx/audio_sample.wav"
            ]
            audio_files = [{"cid": cid, "metadata": {"type": "audio"}, "source": "example"} for cid in example_audio_cids]
            
        print(f"Found {len(audio_files)} audio files on IPFS")
        return audio_files
    
    except Exception as e:
        print(f"Error searching IPFS: {e}")
        return []

def is_audio_by_metadata(metadata):
    """
    Check if a file is an audio file based on its metadata
    
    Args:
        metadata: File metadata dict
        
    Returns:
        Boolean indicating if the file is likely an audio file
    """
    # Check MIME type if available
    mime_type = metadata.get("MimeType", "")
    if mime_type.startswith("audio/"):
        return True
        
    # Check file extension
    name = metadata.get("Name", "")
    audio_extensions = ['.mp3', '.wav', '.ogg', '.flac', '.m4a', '.aac']
    if any(name.lower().endswith(ext) for ext in audio_extensions):
        return True
        
    # Check metadata tags
    tags = metadata.get("Tags", [])
    audio_tags = ["audio", "sound", "music", "recording", "voice", "speech"]
    if any(tag in audio_tags for tag in tags):
        return True
    
    return False

def retrieve_audio_from_ipfs(cid, gateway_url="https://ipfs.io/ipfs"):
    """
    Download audio file from IPFS
    
    Args:
        cid: Content identifier for the IPFS file
        gateway_url: IPFS gateway URL
        
    Returns:
        Path to downloaded file
    """
    try:
        # First try with a local IPFS client
        try:
            client = ipfshttpclient.connect()
            print(f"Retrieving {{cid}} using local IPFS node")
            
            # Create temp file
            fd, temp_path = tempfile.mkstemp(suffix="." + cid.split(".")[-1] if "." in cid else ".mp3")
            os.close(fd)
            
            # Get the file from IPFS and save it
            client.get(cid, target=temp_path)
            return temp_path
            
        except Exception as e:
            print(f"Local IPFS retrieval failed: {e}")
            
            # Fall back to gateway
            file_url = f"{gateway_url}/{cid}"
            print(f"Retrieving {{file_url}} using gateway")
            
            response = requests.get(file_url, stream=True)
            if response.status_code == 200:
                # Create temp file
                fd, temp_path = tempfile.mkstemp(suffix="." + cid.split(".")[-1] if "." in cid else ".mp3")
                with os.fdopen(fd, 'wb') as f:
                    for chunk in response.iter_content(chunk_size=1024):
                        if chunk:
                            f.write(chunk)
                return temp_path
            else:
                print(f"Failed to retrieve file: {response.status_code}")
                return None
    except Exception as e:
        print(f"Error retrieving from IPFS: {e}")
        return None

def process_ipfs_audio_content(query=None, limit=3):
    """
    Main function to search for, retrieve, and process audio content from IPFS
    
    Args:
        query: Optional search query
        limit: Maximum number of files to process
        
    Returns:
        List of processed documents
    """
    print(f"Searching for{''+(' '+query if query else '')} audio content on IPFS...")
    audio_files = search_ipfs_audio_content(query=query, limit=limit)
    
    processed_documents = []
    
    for i, audio_file in enumerate(audio_files):
        print(f"\nProcessing IPFS audio file {{i+1}}/{{len(audio_files)}}: {{audio_file['cid']}}")
        
        # Retrieve the audio file
        local_path = retrieve_audio_from_ipfs(audio_file['cid'])
        if not local_path:
            print("Failed to retrieve audio file")
            continue
        
        try:
            print(f"Downloaded to {{local_path}}, transcribing...")
            
            # Transcribe using our existing Whisper model
            result = model.transcribe(local_path)
            text_content = result["text"]
            
            # Create document
            ipfs_doc = AudioDocument(text_content)
            ipfs_doc.metadata = {
                "source": "ipfs",
                "cid": audio_file['cid'],
                "ipfs_metadata": audio_file['metadata']
            }
            
            # Add to our collection
            processed_documents.append(ipfs_doc)
            
            # Process audio segments
            segments = result["segments"]
            relevant_segments = find_relevant_audio_segments(query if query else "audio", 
                                                            segments, collection, embedding_model)
            
            # Save segments (optional)
            if relevant_segments:
                query_term = query if query else "ipfs_audio"
                save_audio_segments(local_path, relevant_segments, 
                                  f"ipfs_{{sanitize_filename(query_term)}}_{{i}}")
            
            # Clean up the temporary file
            try:
                os.unlink(local_path)
            except:
                pass
                
        except Exception as e:
            print(f"Error processing audio file: {e}")
            # Clean up on error
            try:
                os.unlink(local_path)
            except:
                pass
    
    # If we found and processed any documents, add them to our RAG system
    if processed_documents:
        print(f"\nAdding {{len(processed_documents)}} IPFS audio documents to RAG system...")
        
        # Split documents into chunks
        ipfs_docs = text_splitter.split_documents(processed_documents)
        
        # Attach sound tags (we'll just use generic ones since we don't have the specific audio)
        for doc in ipfs_docs:
            doc.page_content = f"Transcription from IPFS: {{doc.page_content}}\nSource: IPFS CID {{doc.metadata.get('cid', 'unknown')}}"
        
        # Create embeddings
        doc_texts = [doc.page_content for doc in ipfs_docs]
        document_embeddings = embedding_model.encode(doc_texts, convert_to_numpy=True)
        
        # Add to ChromaDB
        start_idx = collection.count()
        for i, embedding in enumerate(document_embeddings):
            collection.add(
                ids=[f"ipfs_{{start_idx + i}}"],
                embeddings=[embedding.tolist()],
                metadatas=[{"text": doc_texts[i], "source": "ipfs"}]
            )
        
        print(f"Successfully added {{len(ipfs_docs)}} IPFS audio document chunks to the RAG system")
        
        # Update metrics
        metrics_tracker.log_embedding_creation(len(document_embeddings))
    else:
        print("No IPFS audio documents were processed")
    
    return processed_documents

# Example usage - try to find audio related to music
# Uncomment to execute
# ipfs_docs = process_ipfs_audio_content(query="music", limit=2)

### Demonstrating IPFS Audio Content Integration

Let's demonstrate how to use the IPFS audio discovery and integration functionality with a simple example. You can customize the search query to find specific types of audio content on IPFS.

In [ ]:
# --- Demo: Integrate IPFS Audio Content with RAG ---

# Search for interview audio on IPFS and process it
ipfs_docs = process_ipfs_audio_content(query="interview", limit=1)

# If we found any documents, test a query that might use this new knowledge
if ipfs_docs:
    print("\n--- Testing RAG with IPFS content ---")
    test_query = "What interviews are available on IPFS?"
    print(f"Query: {test_query}")
    
    # Get context from our RAG system (which now includes IPFS content)
    context = vector_search_tool(test_query)
    print(f"Retrieved context:\n{context}\n")
    
    # Generate a response using our LLM
    prompt = f"""
You are an AI assistant with knowledge about audio content.
Based on the following context, please answer the question: {test_query}

Context:
{context}

Answer:
"""
    
    response = llm(prompt, max_tokens=500)
    print(f"LLM Response:\n{response['choices'][0]['text']}")
else:
    print("\nNo IPFS content was retrieved. You can try again with a different query.")

In [ ]:
# --- Update Metrics Dashboard with IPFS Integration ---

# Extend our RAGMetricsTracker class to include IPFS metrics
class IPFSMetrics:
    def __init__(self):
        self.audio_files_discovered = 0
        self.audio_files_processed = 0
        self.retrieval_errors = 0
        self.processing_errors = 0
        self.last_search_query = ""
        self.last_search_results = 0
    
    def to_dict(self):
        return {
            "audio_files_discovered": self.audio_files_discovered,
            "audio_files_processed": self.audio_files_processed,
            "retrieval_errors": self.retrieval_errors,
            "processing_errors": self.processing_errors,
            "last_search_query": self.last_search_query,
            "last_search_results": self.last_search_results
        }

# Add IPFS metrics to our tracker
metrics_tracker.ipfs_metrics = IPFSMetrics()

# Update the export_metrics method to include IPFS metrics
original_export_metrics = metrics_tracker.export_metrics

def extended_export_metrics():
    metrics = original_export_metrics()
    metrics["ipfs_metrics"] = metrics_tracker.ipfs_metrics.to_dict()
    return metrics

# Replace the original method
metrics_tracker.export_metrics = extended_export_metrics

# Update the dashboard code to display IPFS metrics
dashboard_code = """
import streamlit as st
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import json
import time
import requests

st.set_page_config(page_title="RAG Pipeline Monitor", layout="wide")

st.title("📊 Agentic RAG Pipeline Monitoring")

# Create metrics columns
col1, col2, col3, col4 = st.columns(4)

# Initialize session state
if 'last_update' not in st.session_state:
    st.session_state.last_update = time.time()
    st.session_state.metrics_history = []

# Function to fetch metrics from the metrics endpoint
def get_metrics():
    try:
        response = requests.get(METRICS_API_URL)
        if response.status_code == 200:
            return response.json()
        else:
            st.error(f"Failed to fetch metrics: {response.status_code}")
            return None
    except Exception as e:
        st.error(f"Error fetching metrics: {e}")
        return None

# Set the metrics API endpoint
METRICS_API_URL = "http://127.0.0.1:METRICS_PORT/metrics"

metrics = get_metrics()
if metrics:
    # Store metrics in history
    st.session_state.metrics_history.append(metrics)
    if len(st.session_state.metrics_history) > 20:
        st.session_state.metrics_history.pop(0)
    
    # Display metrics in columns
    with col1:
        st.subheader("Audio Processing")
        st.metric("Files Processed", metrics["audio_metrics"]["processed_files"])
        st.metric("Total Duration (sec)", round(metrics["audio_metrics"]["total_duration"], 2))
        st.metric("Segments Extracted", metrics["audio_metrics"]["segments_extracted"])
    
    with col2:
        st.subheader("ChromaDB Operations")
        st.metric("Total Queries", metrics["chromadb_metrics"]["queries"])
        st.metric("Embeddings Created", metrics["chromadb_metrics"]["embeddings_created"])
        st.metric("Avg Query Time (ms)", round(metrics["chromadb_metrics"]["avg_query_time"] * 1000, 2))
    
    with col3:
        st.subheader("RAG Performance")
        st.metric("User Queries", metrics["rag_metrics"]["total_queries"])
        rag_usage = metrics["rag_metrics"]["context_used"]
        direct_answers = metrics["rag_metrics"]["direct_answers"]
        total = rag_usage + direct_answers
        
        if total > 0:
            rag_percentage = (rag_usage / total) * 100
            st.metric("RAG Usage", f"{rag_percentage:.1f}%")
        else:
            st.metric("RAG Usage", "0%")
    
    # New IPFS metrics column
    with col4:
        st.subheader("IPFS Integration")
        if "ipfs_metrics" in metrics:
            ipfs_metrics = metrics["ipfs_metrics"]
            st.metric("Files Discovered", ipfs_metrics["audio_files_discovered"])
            st.metric("Files Processed", ipfs_metrics["audio_files_processed"])
            
            # Show search info if available
            if ipfs_metrics["last_search_query"]:
                st.text(f"Last search: {ipfs_metrics['last_search_query']}")
                st.text(f"Results: {ipfs_metrics['last_search_results']}")
        else:
            st.text("No IPFS metrics available")
    
    # Time series chart of audio processing
    st.subheader("Audio Processing History")
    if metrics["audio_metrics"]["history"]:
        audio_df = pd.DataFrame(metrics["audio_metrics"]["history"])
        if not audio_df.empty:
            audio_df['timestamp'] = pd.to_datetime(audio_df['timestamp'])
            fig = px.scatter(audio_df, x='timestamp', y='transcription_time', 
                             size='duration', color='segments',
                             title="Audio Transcription Performance")
            st.plotly_chart(fig, use_container_width=True)
    
    # Query visualization
    st.subheader("Recent User Queries")
    if metrics["rag_metrics"]["query_history"]:
        query_df = pd.DataFrame(metrics["rag_metrics"]["query_history"])
        if not query_df.empty:
            query_df['timestamp'] = pd.to_datetime(query_df['timestamp'])
            query_df['used_context_label'] = query_df['used_context'].map({True: "RAG", False: "Direct"})
            
            fig = px.scatter(query_df, x='timestamp', y='response_time',
                            color='used_context_label', hover_data=['query'],
                            title="Query Response Times")
            st.plotly_chart(fig, use_container_width=True)
            
            # Display recent queries in a table
            st.dataframe(query_df[['timestamp', 'query', 'used_context_label', 'response_time']]
                        .rename(columns={'used_context_label': 'Method', 'response_time': 'Time (sec)'}))
    
    st.text(f"Last updated: {time.strftime('%Y-%m-%d %H:%M:%S')}")
    st.text(f"Dashboard port: {dashboard_port}, Metrics API port: {metrics_port}")
    
    # Auto-refresh every 5 seconds
    time.sleep(5)
    st.experimental_rerun()
else:
    st.warning("Failed to fetch metrics data. Make sure the metrics endpoint is running.")
    st.text(f"Looking for metrics at: {METRICS_API_URL}")
    time.sleep(5)
    st.experimental_rerun()
"""

# Write the updated Streamlit app to a new file
with open("rag_dashboard_with_ipfs.py", "w") as f:
    f.write(dashboard_code)

print("Updated dashboard code with IPFS metrics")